In [35]:
import joblib
import csv
import cv2
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix

In [19]:
trained_models_dir = 'trained-models/'
testing_case_dir = 'testing-case-example/Testing/'
testing_case_csv_file_name = 'Testing.csv'

Carregando modelos treinados

In [20]:
svm_model = joblib.load(trained_models_dir + 'svm_model.pkl')

Carregando informações do .CSV com os casos de teste

In [21]:
test_image_names = []
test_image_classes = []

In [22]:
with open(testing_case_dir + testing_case_csv_file_name, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    
    next(csv_reader)
    
    for row in csv_reader:
        test_image_names.append(row[0])
        test_image_classes.append(int(row[1]))

In [23]:
def load_images_from_name_list(base_dir, list_with_names, default_image_width, default_image_height, cv2_imread_colorscheme):
    loaded_images = []
    for image_name in list_with_names:
        im = cv2.imread(base_dir + image_name, cv2_imread_colorscheme)
        im_resized = cv2.resize(im, (default_image_width, default_image_height))
        loaded_images.append(im_resized)
    return loaded_images

In [24]:
test_images = load_images_from_name_list(testing_case_dir, \
                                         test_image_names, \
                                         256, 256, \
                                         cv2.IMREAD_GRAYSCALE)

In [25]:
def pre_process_images_from_list(images_list, \
                                 median_blur_kernel_size, \
                                 clahe_clip_limit, \
                                 clahe_tile_grid_size, \
                                 first_erosion_iterations, \
                                 element_for_erosion_and_dilation, \
                                 first_dilation_iterations, \
                                 addapt_thresh_neighbour_size, addapt_thresh_subtraction_constant):
    
    pre_processed_images = []
    clahe = cv2.createCLAHE(clipLimit = clahe_clip_limit, tileGridSize = clahe_tile_grid_size)
    
    for image in images_list:
        blured_image = cv2.medianBlur(image, median_blur_kernel_size) # aplica filtro de mediana
        clahe_image = clahe.apply(blured_image) # aplica equalizacao de histograma por blocos, uniformiza iluminacao
        min_pixel_value = np.min(clahe_image) # obtem o pixel de menor valor na imagem
        darker_image = clahe_image - min_pixel_value # escurece toda a imagem, subtraindo o valor do menor pixel
        
        erosion_image = cv2.erode(darker_image, \
                                  element_for_erosion_and_dilation, \
                                  iterations = first_erosion_iterations) # aplica 2 erosoes
        
        dilated_image = cv2.dilate(erosion_image, \
                                   element_for_erosion_and_dilation, \
                                   iterations = first_dilation_iterations) # aplica 2 dilatacoes
        
        image_threshold = cv2.adaptiveThreshold(dilated_image, 255, \
                                                cv2.ADAPTIVE_THRESH_MEAN_C, \
                                                cv2.THRESH_BINARY, \
                                                addapt_thresh_neighbour_size, \
                                                addapt_thresh_subtraction_constant)
        
        #image_negative = cv2.bitwise_not(image_threshold)
        pre_processed_images.append(image_threshold)
        
    return pre_processed_images

In [26]:
median_blur_kernel_size = 3
clahe_clip_limit = 4.0
clahe_tile_grid_size = (30, 30)
first_erosion_iterations = 3
first_dilation_iterations = 2
size_for_erosion_and_dilation_element = 2

element_for_erosion_and_dilation = \
    cv2.getStructuringElement(cv2.MORPH_CROSS, \
                              (2 * size_for_erosion_and_dilation_element + 1, 2 * size_for_erosion_and_dilation_element + 1), \
                              (size_for_erosion_and_dilation_element, size_for_erosion_and_dilation_element))

addapt_thresh_neighbour_size = 11
addapt_thresh_subtraction_constant = 2


In [30]:
pre_processed_test_images = pre_process_images_from_list(test_images, \
                                                         median_blur_kernel_size, \
                                                         clahe_clip_limit, \
                                                         clahe_tile_grid_size, \
                                                         first_erosion_iterations, \
                                                         element_for_erosion_and_dilation, \
                                                         first_dilation_iterations, \
                                                         addapt_thresh_neighbour_size, \
                                                         addapt_thresh_subtraction_constant)

Executando modelo SVM

In [31]:
input_masks = list(map(lambda x:x.flatten(), pre_processed_test_images))

In [33]:
svm_predictions = svm_model.predict(input_masks)

In [36]:
accuracy = accuracy_score(svm_predictions, test_image_classes)
accuracy

0.6

In [37]:
conf_matrix = confusion_matrix(svm_predictions, test_image_classes)
conf_matrix

array([[0, 0],
       [4, 6]], dtype=int64)